In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [8]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(schema=TestSchema)
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_20920\2771882717.py:7 in <module>                    │
│                                                                                                  │
│    4   warnings.simplefilter("error")                                                            │
│    5   with cProfile.Profile() as pr:                                                            │
│    6 │   db = DataBase(schema=TestSchema)                                                        │
│ ❱  7 │   rec = await data_source.load([data], db=db)                                             │
│    8 │                                                                                           │
│    9 │   stats = pstats.Stats(pr)                                                                │
│   10                                                                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:848 in load                   │
│                                                                                                  │
│   845 │   │   db = db if db is not None else DataBase()                                          │
│   846 │   │   in_data: InData = {((), ()): dat for dat in data}                                  │
│   847 │   │   rest_data: RestData = []                                                           │
│ ❱ 848 │   │   loaded = await _load_records(db[self.record_type], self, in_data, rest_data)       │
│   849 │   │                                                                                      │
│   850 │   │   # Perform second pass to load remaining data                                       │
│   851 │   │   for table_map, tree_data in rest_data:                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:781 in _load_records          │
│                                                                                                  │
│   778 │   │   │   │   ).items()                                                                  │
│   779 │   │   │   }                                                                              │
│   780 │   │   │                                                                                  │
│ ❱ 781 │   │   │   await _load_records(                                                           │
│   782 │   │   │   │   table.db[rel_map.target.record_type],                                      │
│   783 │   │   │   │   rel_map,                                                                   │
│   784 │   │   │   │   rel_data,                                                                  │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:677 in _load_records          │
│                                                                                                  │
│   674 │   │   │   │   for (_, path_idx), data_item in in_data.items()                            │
│   675 │   │   │   }                                                                              │
│   676 │   │   │                                                                                  │
│ ❱ 677 │   │   │   link_recs = await _load_records(                                               │
│   678 │   │   │   │   table.db[rel.record_type],                                                 │
│   679 │   │   │   │   target_map,                                                                │
│   680 │   │   │   │   link_data,                           

In [ ]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': 'e8b3a8e6-4bf1-4e7f-99ab-48aaef5384c3'}),
    User({'name': 'Jane', 'age': 56, '_id': '374a9007-b76b-45ad-9628-ece988568e42'}),
    User({'name': 'Jack', 'age': 66, '_id': '2825280a-f3fa-4be7-a3ca-c17aea5e5c1d'})
]

In [ ]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results].items())


[
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].df()

test_db_schema_Search_term,test_db_schema_Search_40b57a0db0_test_db_schema_Project_number,end,name,number,status,start,test_db_schema_Organization__id
str,i64,date,str,i64,str,date,str
"""test""",0,2020-01-04,"""baking cake""",0,"""done""",2020-01-01,"""df905981-f18e-4c8b-a328-6cec70…"
"""test""",2,2020-01-06,"""fixing cars""",2,"""started""",2020-01-03,"""d4a76a2c-b771-4e4f-8b46-a0528b…"
"""test""",1,2020-01-05,"""cleaning shoes""",1,"""done""",2020-01-02,"""537a4d40-4b4d-4a66-96bd-1777ab…"


In [14]:
db[Organization].df()

test_db_schema_Organization__id,address,city,_id,name
str,str,str,str,str
"""537a4d40-4b4d-4a66-96bd-1777ab…","""Main Street 1""","""Bakerville""","""537a4d40-4b4d-4a66-96bd-1777ab…","""Bakery"""
"""d4a76a2c-b771-4e4f-8b46-a0528b…","""Main Street 2""","""Shoetown""","""d4a76a2c-b771-4e4f-8b46-a0528b…","""Shoe Shop"""
"""df905981-f18e-4c8b-a328-6cec70…","""Main Street 3""","""Cartown""","""df905981-f18e-4c8b-a328-6cec70…","""Car Shop"""


In [15]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'test_db_schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev133+gf6dad64.d20250116',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '587492bc6b784c9fd5908d64f4e52c9a7294b3b7',
        'module_path': WindowsPath('exp/test_db_schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'test_db_schema.User': 3,
            'test_db_schema.Task': 6,
            'test_db_schema.Search': 1,
            'test_db_schema.Organization': 3,
            'test_db_schema.Project': 3
        },
        'arrays': {'test_db_schema_Organization_countries': 0},
        'relations': {'test_db_schema.Membership': 7, 'test_db_schema.SearchResult': 3}
    },
    'backend': None
}

In [ ]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

test_db_schema_SearchResult__id,score,_id,test_db_schema_Project_number,test_db_schema_Search_term
str,f64,str,i64,str
"""6255212236""",0.9,"""6255212236""",0,"""test"""


In [ ]:
db[Organization.countries].df()

test_db_schema_Organization__id,test_db_schema_Organization_37fc052c3c__id,_value
null,null,null


In [24]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         558526 function calls (521929 primitive calls) in 1.171 seconds

   Ordered by: cumulative time
   List reduced from 2274 to 145 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.000    0.000    0.518    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_rec_from_item)
       52    0.002    0.000    0.516    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:567(_load_record)
       23    0.000    0.000    0.409    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1450(__ior__)
       23    0.001    0.000    0.408    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2886(_mutate)
       56    0.000    0.000    0.382    0.007 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:829(load)
   168/57    0.003    0.000    0.352    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py